In [12]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from scipy import stats

In [13]:
categories = ['banana','beach','car','church','lighthouse','mountain','mug','plane']
path='../Analysis/human_rating_bycat'

In [14]:
for i, cate in enumerate(categories):
    col = [cate+'_'+str(i+1) for i in range(16)]
    new = pd.read_csv(path+'/'+cate+'.csv')
    new.columns = col
    if i == 0:
        old = new
    else:
        old = pd.concat([old,new],axis=1)
df = old

(Test) Use quantiles to remove outliers 

In [15]:
quantile_low = df.quantile(0.1)
quantile_high = df.quantile(0.9)
iqr = quantile_high - quantile_low

In [16]:
outliers = (df < quantile_low) | (df > quantile_high)
df_new = df[outliers.sum(axis=1) <= 12]

Compute 7 scorse and sort images by order in survey

In [17]:
googlenet_df=pd.read_csv('googlenet_scores_wordnet_id_closest_label.csv',index_col=0)
googlenet_df['typicality_score'] = np.round(googlenet_df['closest_prob'] * 100,2)
googlenet_df['7_score']= np.round(googlenet_df['typicality_score']*7/100,2)
googlenet_df['type'] = googlenet_df['type'].apply(lambda x: x.lower())

In [18]:
googlenet_df['number'] = googlenet_df['dir'].apply(lambda x: int(x.split('\\')[-1].split('.')[0]))
googlenet_df = googlenet_df.sort_values(['type','number']).reset_index(drop=True)

# Correlation with human ratings

Changes from original script: 
1. Use spearman rank correlation instead of pearson
2. Compute average human ratings first, and then compute correlation with neural net typicality scores

without dropping outliers

In [19]:
final_corr = dict()
for t in np.unique(googlenet_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    human_ratings = df[col_names] 
    net_ratings =googlenet_df[googlenet_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
#     net_scores = net_ratings['typicality_score'].values
    
    coef, p=stats.spearmanr(human_scores, net_scores)
#     coef, p=stats.pearsonr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.31935255150510367
beach:  0.3058823529411765
car:  0.3399559419247878
church:  0.22328672238187228
lighthouse:  0.34705882352941175
mountain:  0.4709346381642081
mug:  -0.004418266941843933
plane:  0.5504048583544183


In [20]:
print(np.mean(list(final_corr.values())))

0.3190572027323918


after dropping outliers

In [10]:
final_corr = dict()
for t in np.unique(googlenet_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    
    human_ratings = df_new[col_names] 
    net_ratings = googlenet_df[googlenet_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
    coef, p=stats.spearmanr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.33406925894773515
beach:  0.36377064487848376
car:  0.29624179009080087
church:  0.22844516151280664
lighthouse:  0.3431520658165455
mountain:  0.41513611441714093
mug:  0.044247980125809834
plane:  0.43952993591637773


In [11]:
print(np.mean(list(final_corr.values())))

0.3080741189632126
